In [ ]:
!unzip --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O /tmp/bbc-text.csv

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2022-12-12 07:41:05--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.31.128, 74.125.134.128, 74.125.139.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.04s   

2022-12-12 07:41:05 (134 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [ ]:
import csv
import numpy as np
import pandas as pd

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
stopword_list = set(stopwords.words('english'))
print(stopword_list)

{'hasn', 'each', "wasn't", 'shouldn', 'but', 'was', 'weren', 'those', 'the', 'himself', "isn't", 'after', "don't", 'why', 'just', 'than', 'same', 'them', 'not', 'as', 'on', 'd', 'it', "you're", 'from', 's', 'we', 'out', 'y', 'his', 'ain', 'does', 'a', 'theirs', 'wouldn', 'who', 'to', 'yours', "should've", 'mustn', 'what', 'its', 'is', 'you', 'doing', 'll', 'he', 'below', 't', 'needn', 'if', 'now', 'down', 'during', 'wasn', 'because', "couldn't", 'haven', 'these', 'off', 'have', 'ma', 'between', "you'd", 'him', 'before', 'being', 'i', 'should', 'this', 'up', 'until', 'any', 'other', "weren't", 'there', 'their', 'when', "aren't", 'by', "shouldn't", 'didn', "you'll", 'into', 'will', 'through', 'while', 're', 'here', 'that', 'own', 'myself', 'couldn', 'mightn', 'further', "won't", 'which', 'or', 'of', 'few', 'no', "hasn't", 'for', 'hadn', 'most', 'shan', 'all', 'with', "doesn't", 'whom', 'in', "didn't", 'don', 'having', 'nor', 'over', 'above', 'such', 'only', 'once', 'me', 'an', 'under', "

In [ ]:
news_text = []
news_topics = []

with open("/tmp/bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        news_topics.append(row[0])
        t = row[1]
        lowered = t.lower()
        # remove numbers 
        no_numbers = re.sub(pattern=r'\d+', repl=" ", string=lowered)
        # remove punctuations 
        no_punct = no_numbers.translate(str.maketrans('', '', string.punctuation))
        # remove stopwords 
        for term in stopword_list:
            token = ' ' + term + ' '
            no_stopwords = no_punct.replace(token, ' ')
            no_stopwords = no_stopwords.replace(' ', ' ')
        # remove whitespace (more than single space)
        no_ws = re.sub(pattern=r'\s+[a-zA-Z]\s+', repl=" ", string=no_stopwords)
        # lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = word_tokenize(no_ws)
        for i in range(len(tokens)):
            lemma_word = lemmatizer.lemmatize(tokens[i])
            tokens[i] = lemma_word
        text = " ".join(tokens)
        news_text.append(text)

In [ ]:
df = pd.DataFrame()
df['Text'] = news_text
df['Topic'] = news_topics
df.head()

,Text,Topic
0,tv future in the hand of viewer with home thea...,tech
1,worldcom bos left book alone former worldcom b...,business
2,tiger wary of farrell gamble leicester say the...,sport
3,yeading face newcastle in fa cup premiership s...,sport
4,ocean twelve raid box office ocean twelve the ...,entertainment


In [ ]:
df['Topic'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: Topic, dtype: int64

In [ ]:
news_train, news_test, topic_train, topic_test = train_test_split(df['Text'], df['Topic'], test_size=0.2, random_state=42, shuffle=True, stratify=df['Topic'])

In [ ]:
text_tokenizer = Tokenizer(num_words = 5000, oov_token='<OOV>')
text_tokenizer.fit_on_texts(news_train)
print(text_tokenizer.word_index)

{'<OOV>': 1, 'the': 2, 'to': 3, 'of': 4, 'and': 5, 'in': 6, 'it': 7, 'for': 8, 'is': 9, 'that': 10, 'on': 11, 'said': 12, 'wa': 13, 'he': 14, 'be': 15, 'with': 16, 'a': 17, 'ha': 18, 'have': 19, 'at': 20, 'will': 21, 'by': 22, 'are': 23, 'but': 24, 'not': 25, 'from': 26, 'they': 27, 'we': 28, 'mr': 29, 'his': 30, 'year': 31, 'this': 32, 'an': 33, 'would': 34, 'which': 35, 'had': 36, 'been': 37, 'more': 38, 'their': 39, 'also': 40, 'who': 41, 'people': 42, 'up': 43, 'new': 44, 'u': 45, 'there': 46, 'can': 47, 'about': 48, 'one': 49, 'than': 50, 'out': 51, 'after': 52, 'or': 53, 'all': 54, 'if': 55, 'you': 56, 'could': 57, 'time': 58, 'last': 59, '£': 60, 'game': 61, 'over': 62, 'when': 63, 'say': 64, 'first': 65, 'government': 66, 'world': 67, 'two': 68, 'company': 69, 'other': 70, 'some': 71, 'what': 72, 'into': 73, 'now': 74, 'film': 75, 'make': 76, 'uk': 77, 'so': 78, 'she': 79, 'firm': 80, 'do': 81, 'no': 82, 'them': 83, 'against': 84, 'only': 85, 'bn': 86, 'get': 87, 'just': 88, 'b

In [ ]:
train_sequences = text_tokenizer.texts_to_sequences(news_train)
validation_sequences = text_tokenizer.texts_to_sequences(news_test)
train_padded = pad_sequences(train_sequences, maxlen=200, padding='post', truncating='post')
validation_padded = pad_sequences(validation_sequences, maxlen=200, padding='post', truncating='post')

In [ ]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(news_topics)
training_label_seq = np.array(label_tokenizer.texts_to_sequences(topic_train))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(topic_test))

In [ ]:
print(label_tokenizer.word_index)

{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}


In [ ]:
model = Sequential([Embedding(5000,16),
                    Dropout(0.5),  
                    Bidirectional(LSTM(64)),
                    Dense(128, activation='relu'),
                    Dropout(0.5), 
                    Dense(64, activation='relu'),
                    Dense(6, activation='softmax')])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001, decay=1e-6), metrics=['accuracy'])
model.fit(train_padded, training_label_seq, epochs=7, validation_data=(validation_padded, validation_label_seq), verbose=1)

Epoch 1/7
56/56 [==============================] - 16s 205ms/step - loss: 1.6779 - accuracy: 0.2331 - val_loss: 1.5955 - val_accuracy: 0.2292
Epoch 2/7
56/56 [==============================] - 10s 183ms/step - loss: 1.5495 - accuracy: 0.3051 - val_loss: 1.5024 - val_accuracy: 0.4270
Epoch 3/7
56/56 [==============================] - 13s 226ms/step - loss: 1.1414 - accuracy: 0.4798 - val_loss: 0.8853 - val_accuracy: 0.5955
Epoch 4/7
56/56 [==============================] - 12s 217ms/step - loss: 0.5883 - accuracy: 0.7545 - val_loss: 0.4961 - val_accuracy: 0.8337
Epoch 5/7
56/56 [==============================] - 11s 188ms/step - loss: 0.2904 - accuracy: 0.8978 - val_loss: 0.3574 - val_accuracy: 0.8966
Epoch 6/7
56/56 [==============================] - 10s 187ms/step - loss: 0.1517 - accuracy: 0.9545 - val_loss: 0.2452 - val_accuracy: 0.9124
Epoch 7/7
56/56 [==============================] - 10s 186ms/step - loss: 0.0998 - accuracy: 0.9702 - val_loss: 0.2717 - val_accuracy: 0.9213
